In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
import re
import os

In [8]:
def tally_occurrences(df_list):
    counter = Counter()
    
    for word in df_list:
        counter[word] += 1
    
    return counter

In [9]:
def last_statement_counter(filepath):
    counter = Counter()
    
    df = pd.read_csv(filepath, encoding="iso-8859-1")
    df["word_list"] = df["LastStatement"].str.replace(r"[,\/.:\"?]", "").apply(lambda x: x.split())
    # df["word_list"] = df["word_list"].apply(lambda x: x.split())
    df["word_counter"] = df["word_list"].apply(lambda x: tally_occurrences(x))
    df["most_common_words"] = df["word_list"].apply(lambda x: Counter(x).most_common(10))
    
    return df

In [10]:
texas_last_df = last_statement_counter("./Texas Last Statement.csv"); texas_last_df

,Execution,LastName,FirstName,TDCJNumber,Age,Race,CountyOfConviction,AgeWhenReceived,EducationLevel,NativeCounty,...,WhiteVictim,HispanicVictim,BlackVictim,VictimOther Races,FemaleVictim,MaleVictim,LastStatement,word_list,word_counter,most_common_words
0,545,Cardenas,Ruben,999275,47,Hispanic,Hidalgo,28.0,11.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,"This is my statement: My final words. First, I...","[This, is, my, statement, My, final, words, Fi...","{'This': 1, 'is': 1, 'my': 3, 'statement': 1, ...","[(I, 8), (for, 6), (to, 5), (me, 5), (and, 5),..."
1,544,Pruett,Robert,999411,38,White,Bee,22.0,8.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,I just want to let everyone in here to know I ...,"[I, just, want, to, let, everyone, in, here, t...","{'I': 4, 'just': 1, 'want': 1, 'to': 5, 'let':...","[(to, 5), (I, 4), (love, 3), (so, 3), (much, 3..."
2,543,Preyor,Taichin,999494,46,Black,Bexar,34.0,10.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,"First and foremost I'd like to say, ""Justice h...","[First, and, foremost, I'd, like, to, say, Jus...","{'First': 1, 'and': 3, 'foremost': 1, 'I'd': 1...","[(and, 3), (to, 3), (by, 2), (my, 2), (First, ..."
3,542,Bigby,James,997,61,White,Tarrant,36.0,9.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,"Yes, I do, Grace Kehler is that you? I have gi...","[Yes, I, do, Grace, Kehler, is, that, you, I, ...","{'Yes': 1, 'I': 12, 'do': 1, 'Grace': 2, 'Kehl...","[(I, 12), (you, 10), (I'm, 8), (sorry, 8), (th..."
4,541,Ruiz,Rolando,999145,44,Hispanic,Bexar,22.0,10.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,"Yes sir, I would first like to say to the Sanc...","[Yes, sir, I, would, first, like, to, say, to,...","{'Yes': 1, 'sir': 1, 'I': 7, 'would': 1, 'firs...","[(I, 7), (you, 5), (and, 4), (to, 3), (am, 3),..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,5,Skillern,Doyle,518,49,White,Lubbock,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,I pray that my family will rejoice and will fo...,"[I, pray, that, my, family, will, rejoice, and...","{'I': 1, 'pray': 1, 'that': 1, 'my': 1, 'famil...","[(will, 2), (I, 1), (pray, 1), (that, 1), (my,..."
541,4,Barefoot,Thomas,621,39,White,Bell,33.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,"When asked if he had a last statement, he repl...","[When, asked, if, he, had, a, last, statement,...","{'When': 1, 'asked': 1, 'if': 1, 'he': 3, 'had...","[(I, 6), (to, 5), (that, 4), (the, 4), (he, 3)..."
542,3,O'Bryan,Ronald,529,39,White,Harris,31.0,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,1.0,What is about to transpire in a few moments is...,"[What, is, about, to, transpire, in, a, few, m...","{'What': 1, 'is': 4, 'about': 1, 'to': 3, 'tra...","[(I, 8), (in, 6), (and, 5), (all, 5), (my, 5),..."
543,2,Autry,James,670,29,White,Jefferson,26.0,6.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,None,[None],{'None': 1},"[(None, 1)]"


In [11]:
def create_list_of_column_names(string, column_number_list):
    counter = 0
    column_name_list = []
    
    for item in column_number_list:
        counter+=1
        column_name_list.append("{}_{}".format(string, counter))
    
    return column_name_list

In [12]:
def get_word_aggregation(word_df):
    word_df_list = word_df.values.tolist()
    word_df_aggregation = []

    for item in word_df_list:
        for word in item:
            if type(word) != float and word != "None":
                word_df_aggregation.append(word)
    
    return word_df_aggregation

In [ ]:
""" Tests

a_list_one = [("I", 8), ("for", 6), ("to", 5), ("me", 5), ("and", 5)]
words, occurrences = [list(item) for item in zip(*a_list_one)]; words
updated_df = pd.DataFrame(texas_last_df["most_common_words"].str.split().values.tolist()); updated_df
temp_df_one = pd.DataFrame.from_records(a_list_one, columns=["word", "occurrence"]); temp_df_one

"""

### TOP NUMBER OF VICTIMS (subset)
Which person has the highest number of victims?

In [ ]:
top_number_of_victim = texas_last_df[texas_last_df["NumberVictim"] >= 3]; top_number_of_victim

In [ ]:
occurrences_unzip = top_number_of_victim["most_common_words"].apply(lambda x: [list(item) for item in zip(*x)])

In [ ]:
occurrences_df = pd.DataFrame(occurrences_unzip.apply(pd.Series))
occurrences_df.columns = ["word", "occurrences"]

In [ ]:
words_df = occurrences_df["word"].apply(pd.Series)
words_df.columns = create_list_of_column_names("word", words_df)

In [ ]:
occurrences_number_df = occurrences_df["occurrences"].apply(pd.Series).fillna(0).apply(lambda x: x.astype(int))
occurrences_number_df.columns = create_list_of_column_names("occurrences_number", occurrences_number_df)

In [ ]:
combined_word_occurrences_df = pd.concat([words_df, occurrences_number_df], axis=1, sort=False)

### TOP VICTIMS WORD OCCURRENCES AGGREGATION
Parse rows to find most common words used overall

In [ ]:
top_word_aggregation = get_word_aggregation(words_df)
top_word_counter = Counter(top_word_aggregation).most_common(20)
offenders_with_five_victims = pd.DataFrame.from_records(top_word_counter, columns=["word", "occurrence"]); offenders_with_five_victims

### ALL OFFENDERS
Checkout all offenders

In [13]:
all_offenders = texas_last_df

In [14]:
occurrences_unzip = all_offenders["most_common_words"].apply(lambda x: [list(item) for item in zip(*x)])

In [15]:
all_occurrences_df = pd.DataFrame(occurrences_unzip.apply(pd.Series))
all_occurrences_df.columns = ["word", "occurrences"]

In [16]:
all_words_df = all_occurrences_df["word"].apply(pd.Series)
all_words_df.columns = create_list_of_column_names("word", all_words_df)

In [17]:
all_word_aggregation = get_word_aggregation(all_words_df)
all_word_counter = Counter(all_word_aggregation).most_common(20)
all_offenders_aggregation_df = pd.DataFrame.from_records(all_word_counter, columns=["word", "occurrence"]); all_offenders_aggregation_df

,word,occurrence
0,I,393
1,to,292
2,you,236
3,and,221
4,the,189
5,my,147
6,for,131
7,love,123
8,all,106
9,that,98


### FINAL TOP VICTIMS DATAFRAMES
Concatenate Top Victim and word occurrence collection into one dataframe

In [ ]:
top_occurrences_df = pd.concat([top_number_of_victim, combined_word_occurrences_df,], axis=1, sort=False); top_occurrences_df

In [ ]:
texas_last_cleaned_df.to_csv("texas_last_statement_cleaned.csv ", index=False)

In [ ]:
all_offenders_aggregation_df.to_csv("all_offenders.csv", index=False)